In [1]:
! curl -s https://apertium.projectjj.com/apt/install-nightly.sh | sudo bash
! sudo apt-get install hfst lexd
! lexd 08_example.lexd | hfst-txt2fst | hfst-invert | hfst-fst2fst -O -o analyzer.hfstol
! cat 08_corpus.txt | hfst-proc -C analyzer.hfstol > analysis.txt

In [4]:
import pandas as pd
import re

In [5]:
df_gold = pd.read_csv('08_gold_standard.csv')
df_gold['tags'] = df_gold['tags'].apply(lambda x: set(x.replace(' ','').split(',')) if str(x) !='nan' else set())
df_gold['tags'] = df_gold['tags'].apply(lambda x: '_'.join(sorted(x)))
df_gold['full'] = df_gold['stem'] + '-' + df_gold['pos'] + '-' + df_gold['tags']
df_gold

,token_id,token,stem,pos,tags,full
0,1,я,я,PRON,nom,я-PRON-nom
1,2,пью,пить,V,1p_npst_sg,пить-V-1p_npst_sg
2,3,ты,ты,PRON,nom,ты-PRON-nom
3,4,пьёшь,пить,V,2p_npst_sg,пить-V-2p_npst_sg
4,5,мы,мы,PRON,nom,мы-PRON-nom
5,6,пьём,пить,V,1p_npst_pl,пить-V-1p_npst_pl
6,7,вы,вы,PRON,nom,вы-PRON-nom
7,8,пьёте,пить,V,2p_npst_pl,пить-V-2p_npst_pl
8,9,спрягал,спрягать,V,m_pst_sg,спрягать-V-m_pst_sg
9,10,глагол,глагол,N,acc,глагол-N-acc


In [6]:
with open('analysis.txt') as f:
    text = f.read()
    text += '\n'

analysis = {}
words = re.findall('''"<(.*?)>"\n((\t.*?\n)+)''', text)
for id, word in enumerate(words):
    stem_s = set()
    pos_s = set()
    tags_s = set()
    full_s = set()
    if not word[1].startswith('\t"*'):
        for razbor in word[1].strip('\n').split('\n'):
            stem, pos_tags = razbor.replace('"', '').strip('\t').split('\t')
            pos_tags_split = pos_tags.split(' ')
            pos = pos_tags_split[0]
            if len(pos_tags_split) > 1:
                tags = '_'.join(sorted(set(pos_tags_split[1:])))
            else:
                tags = ''
            full = stem + '-' + pos  + '-' + tags
            stem_s.add(stem)
            pos_s.add(pos)
            tags_s.add(tags)
            full_s.add(full)

    analysis[id] = {'token': word[0],
                    'stem': stem_s,
                    'pos': pos_s,
                    'tags': tags_s,
                    'full': full_s}

In [7]:
analysis

{0: {'token': 'я',
  'stem': {'я'},
  'pos': {'PRON'},
  'tags': {'nom'},
  'full': {'я-PRON-nom'}},
 1: {'token': 'пью',
  'stem': set(),
  'pos': set(),
  'tags': set(),
  'full': set()},
 2: {'token': 'ты',
  'stem': {'ты'},
  'pos': {'PRON'},
  'tags': {'nom'},
  'full': {'ты-PRON-nom'}},
 3: {'token': 'пьёшь',
  'stem': set(),
  'pos': set(),
  'tags': set(),
  'full': set()},
 4: {'token': 'мы',
  'stem': {'мы'},
  'pos': {'PRON'},
  'tags': {'nom'},
  'full': {'мы-PRON-nom'}},
 5: {'token': 'пьём',
  'stem': set(),
  'pos': set(),
  'tags': set(),
  'full': set()},
 6: {'token': 'вы',
  'stem': {'вы'},
  'pos': {'PRON'},
  'tags': {'nom'},
  'full': {'вы-PRON-nom'}},
 7: {'token': 'пьёте',
  'stem': set(),
  'pos': set(),
  'tags': set(),
  'full': set()},
 8: {'token': 'спрягал',
  'stem': set(),
  'pos': set(),
  'tags': set(),
  'full': set()},
 9: {'token': 'глагол',
  'stem': {'глагол'},
  'pos': {'N'},
  'tags': {'acc_sg', 'nom_sg'},
  'full': {'глагол-N-acc_sg', 'глагол-N

In [8]:
# принимает на вход название столбика таблицы (по чему считаются метрики)
# выдаёт precision, recall, f1-меру, посчитанные двумя способами

def get_metrics(an_part):

    # для подсчёта средних от метрик
    # (считаем для каждого слова отдельно, а потом берём среднее)
    precision_s = []
    recall_s = []
    f1_s = []

    # для подсчёта общих метрик
    # считаем tp(good_s), fp(bad_s) и fn(not_found_s) и в конце считаем метрики
    good_s = 0
    bad_s = 0
    not_found_s = 0

    for id, row in df_gold.iterrows():
        gold = row[an_part]
        pred = analysis[id][an_part]

        good = 1 if gold in pred else 0  # наличие правильного предсказания
        bad = len(pred - {gold})  # кол-во неправильных предсказаний
        not_found = len({gold}) - good  # кол-во того, что не предсказали (хотя должны были)
        good_s += good
        bad_s += bad
        not_found_s += not_found

        if len(pred) != 0:
            precision = good / len(pred)  # good/(good+bad)
        else:
            precision = 1

        if len({gold}) != 0:
            recall = good / len({gold})  # good/(good+not_found)
        else:
            recall = 1
        if precision+recall != 0:
            f1 = 2*(precision*recall)/(precision+recall)
        else:
            f1 = 0

        precision_s.append(precision)
        recall_s.append(recall)
        f1_s.append(f1)

    mean_precision = sum(precision_s)/len(precision_s)
    mean_recall = sum(recall_s)/len(recall_s)
    mean_f1 = sum(f1_s)/len(f1_s)
    fin_precision = good_s/(good_s+bad_s)
    fin_recall = good_s/(good_s+not_found_s)
    fin_f1 = 2*(fin_precision*fin_recall)/(fin_precision+fin_recall)

    return {'mean_precision': mean_precision,
            'mean_recall': mean_recall,
            'mean_f1': mean_f1,
            'fin_precision': fin_precision,
            'fin_recall': fin_recall,
            'fin_f1': fin_f1}

In [9]:
for an_part in ['stem', 'pos', 'tags', 'full']:
    print(an_part)
    print(get_metrics(an_part))

stem
{'mean_precision': 1.0, 'mean_recall': 0.6470588235294118, 'mean_f1': 0.6470588235294118, 'fin_precision': 1.0, 'fin_recall': 0.6470588235294118, 'fin_f1': 0.7857142857142858}
pos
{'mean_precision': 1.0, 'mean_recall': 0.6470588235294118, 'mean_f1': 0.6470588235294118, 'fin_precision': 1.0, 'fin_recall': 0.6470588235294118, 'fin_f1': 0.7857142857142858}
tags
{'mean_precision': 0.8235294117647058, 'mean_recall': 0.5294117647058824, 'mean_f1': 0.4901960784313726, 'fin_precision': 0.6428571428571429, 'fin_recall': 0.5294117647058824, 'fin_f1': 0.5806451612903226}
full
{'mean_precision': 0.8235294117647058, 'mean_recall': 0.5294117647058824, 'mean_f1': 0.4901960784313726, 'fin_precision': 0.6428571428571429, 'fin_recall': 0.5294117647058824, 'fin_f1': 0.5806451612903226}
